In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_490441/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.1)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_47_2,0.999889,0.825928,0.814037,0.999601,0.000463,1.164022,0.665178,0.000971,0.061682,0.021518,1.007119,0.021857,145.355529,286.690702,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_47_5,0.999884,0.825446,0.813656,0.999411,0.000483,1.167243,0.666540,0.001433,0.060597,0.021978,1.007427,0.022324,145.270913,286.606086,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_47_3,0.999883,0.825743,0.814043,0.999498,0.000489,1.165257,0.665156,0.001221,0.064482,0.022105,1.007513,0.022453,145.247750,286.582922,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_47_1,0.999883,0.826225,0.814004,0.999673,0.000489,1.162033,0.665297,0.000797,0.063234,0.022114,1.007519,0.022462,145.246211,286.581384,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_47_6,0.999882,0.825364,0.813538,0.999315,0.000492,1.167790,0.666965,0.001666,0.058002,0.022181,1.007565,0.022530,145.234052,286.569225,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
467,model_1_3_2,0.898658,0.767111,0.693063,0.899774,0.421839,1.557332,1.097894,0.550995,1.150374,0.649491,7.485886,0.659720,131.726265,273.061437,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
468,model_1_3_1,0.896295,0.765777,0.690886,0.897380,0.431676,1.566253,1.105683,0.564158,1.163195,0.657020,7.637135,0.667368,131.680161,273.015333,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
469,model_1_3_0,0.893280,0.764609,0.686784,0.894534,0.444224,1.574062,1.120355,0.579806,1.182494,0.666501,7.830069,0.676998,131.622852,272.958025,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
470,model_1_2_1,0.884416,0.754893,0.691027,0.883282,0.481122,1.639028,1.105178,0.408619,1.167433,0.693630,8.397387,0.704554,131.463268,272.798440,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
